# CombineSampleTexts

I endeavor to create a file called `ces.training_text` containing the concatenated content of all old Assyrian sample texts that are available in our collection.

In [36]:
from numpy.random import default_rng
import re

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/tesstrain/
!ls

/content/drive/.shortcut-targets-by-id/1W2EROe2FItlaK99U-WY_qaBOc2UD_LI0/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/tesstrain
cdliatf_unblocked.atf	  fontlist-ugly.txt  OATP_published.txt    output
CombineSampleTexts.ipynb  jsonzip	     OATP_unpublished.txt  README.md


## Processing the UR III Dataset

In [108]:
def random_substring(text: str, length: int, rng = default_rng()) -> str:
  """Returns a substring of length `length` selected uniformly at random from
  `text`. (This is uniform in the sense that all possible start and end points
  are selected with equal probability, not in the sense that all text is
  contained in the returned substring with equal probability.)
  :param text: an arbitrary string
  :param length: the length of the desired substring
  :param rng: a numpy Generator instance
  """
  assert len(text) >= length, ("text is not long enough to contain a substring "
                              "of the desired length.")
  start = rng.integers(0, len(text) - length)
  return text[start:start+length]

In [132]:
def process_oatp(oatp: str) -> str:
  """Returns `oatp` with alterations intended to improve OCR training outcomes.
  """
  # Prevent huge strings of text from all being on one line
  ret = oatp.replace('[\t]', '[  ]').replace('\t', '\n')
  ret = re.sub(r'@\S*', '', ret) # remove annotations such as @END_FILE
  ret = re.sub(r'(^|\s)\\\S+', '\n', ret) # remove annotations such as \documentstyle
  ret = re.sub(r'\n+\s*', '\n', ret) # remove redundant newlines
  return ret

In [134]:
with open('OATP_published.txt', 'r') as published:
  with open('OATP_unpublished.txt', 'r') as unpublished:
    oatp = published.read() + unpublished.read()
print(random_substring(process_oatp(oatp), 1000))

a A-šùr-ba-ni 
[
K]B-áp-šu 
[
]x-ú-ni-ma 
[  ] i-zu-zu 10 E
[
ITU.KAM ṣ]í-ib-e-em R
[li-mu-u]m A-šùr-ANDÙL 
[DUMU
]-A-šùr IGI Ha-na-a 
[DUMU A-šur-m]u-ša-lim 
[IGI
] DUMU A-šur-mu-ša-lim     Kt f/k 35 #6 13: Balkan, AS 23, 4A.5 ##  
13 Aššur-idi     Kt f/k 41  1
a-na E-me-me 
qí-bi-ma um-ma 
Ma-nu-um-ba-lúm-A-šur-ma 
5 GÍN KÙ.GI 5
ku-pu-ur-ší-nim 
a-na ba-ri-ni 
šé-bi-lam-ma 
iš-tí . Lu-bu-uš-ta-i-e 
ú-ṭá-tám 10
la-áš-a-am 
la-ma . ú-ni-ší-šu-nu 
ar-hi-iš 
šé-bi-lam-ma 
ri-iš-šu-nu 15
lu-kà-il5 
iš-tí ša-ni-im 
lá i-lá-qé-ú     Kt f/k 49 l.16-20: Albayrak 2003, Erciyes Üniversitesi Kayseri ve Yöresi Tarih Arasrirmalari Merkizi Yayin No. 6  1
a-na Pè-ru-a qí-bi4-ma 
um-ma A-mur-A-šùr-ma 
30 x -ri-im 
i-na ší-mì-ni e-dí-mu-/ni-a-tí 5
lá-ma UDU.HI.A 
lá ni-ib-qú-un 
ni-ba-qá-an-ma 
Šu-Be-lim a-ṭá-ra-dam 
KB 1 MA.NA ṣa-ru-pá-am 10
x -ni-šu 11-12
ú 5 DUG ṭá-ab-tum li-ib-ší 
1 SÌLA Ì.GIŠ 
ú-šé-ba-lam-ma 15
a-ta . ú a-wi-il5-tí 
zu-za . šu-ma a-na 
a-kà-al ṣú-ha-ri 
lá-šu 1 DUG ŠE 
dí-in mì-m

In [136]:
with open('oatp_processed.txt', 'w') as out:
  out.write(process_oatp(oatp))

## Accessing the ORACC Dataset

In [ ]:
!wget https://raw.githubusercontent.com/cdli-gh/data/master/cdliatf_unblocked.atf

--2021-07-18 01:41:47--  https://raw.githubusercontent.com/cdli-gh/data/master/cdliatf_unblocked.atf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83581625 (80M) [text/plain]
Saving to: ‘cdliatf_unblocked.atf’

cdliatf_unblocked.a 100%[===================>]  79.71M  38.0MB/s    in 2.1s    

2021-07-18 01:41:50 (38.0 MB/s) - ‘cdliatf_unblocked.atf’ saved [83581625/83581625]



In [ ]:
!ls

cdliatf_unblocked.atf	  fontlist-ugly.txt   OATP_unpublished.txt
CombineSampleTexts.ipynb  OATP_published.txt  README.md


In [ ]:
#create necessary directories
import os
os.makedirs('jsonzip', exist_ok=True)
os.makedirs('output', exist_ok=True)

# import necessary libraries
import pandas as pd
from tqdm.auto import tqdm

# import libraries for this section
import requests
import zipfile
import json

In [ ]:
project = 'epsd2/admin/ur3' #define project to download.

CHUNK = 1024 #define chunk size

proj = project.replace('/', '-')
url = "http://build-oracc.museum.upenn.edu/json/%s.zip" % proj
file_name = "jsonzip/%s.zip" % proj
with requests.get(url, stream=True) as request:
    if request.status_code == 200:
        #if file is accessible, download
        tqdm.write('Saving ' + url + ' as ' + file_name)
        total_size = int(request.headers.get('content-length', 0))
        tqdm_handler = tqdm(total=total_size, unit='B', unit_scale=True, desc = project)
        with open(file_name, 'wb') as zip_file:
        #use tdqm to show download speed
            for chunk in request.iter_content(chunk_size=CHUNK):
                tqdm_handler.update(len(chunk))
                zip_file.write(chunk)
    else:
        tqdm.write("WARNING: %s does not exist." % url)

Saving http://build-oracc.museum.upenn.edu/json/epsd2-admin-ur3.zip as jsonzip/epsd2-admin-ur3.zip


In [ ]:
result = requests.get('http://oracc.org/projects.json')
result

<Response [200]>

In [ ]:
print(result.json())

{'type': 'projects', 'public': ['adsd', 'adsd/adart1', 'adsd/adart2', 'adsd/adart3', 'adsd/adart6', 'aemw', 'aemw/alalakh/idrimi', 'aemw/amarna', 'aemw/ugarit', 'akklove', 'amgg', 'ario', 'armep', 'arrim', 'asbp', 'asbp/ninmed', 'asbp/rlasb', 'atae', 'atae/assur', 'atae/burmarina', 'atae/durkatlimmu', 'atae/guzana', 'atae/huzirina', 'atae/imgurenlil', 'atae/kalhu', 'atae/mallanate', 'atae/marqasu', 'atae/nineveh', 'atae/samal', 'atae/szibaniba', 'atae/tilbarsip', 'blms', 'btto', 'cams', 'cams/akno', 'cams/anzu', 'cams/barutu', 'cams/etana', 'cams/gkab', 'cams/ludlul', 'cams/selbi', 'cams/tlab', 'caspo', 'caspo/akkpm', 'ccpo', 'cdli', 'ckst', 'cmawro', 'cmawro/cmawr1', 'cmawro/cmawr2', 'cmawro/cmawr3', 'cmawro/maqlu', 'contrib', 'contrib/amarna', 'contrib/lambert', 'ctij', 'dcclt', 'dcclt/ebla', 'dcclt/jena', 'dcclt/nineveh', 'dcclt/signlists', 'dccmt', 'doc', 'dsst', 'ecut', 'epsd2/issl', 'etcsri', 'glass', 'hbtin', 'issl', 'lacost', 'lovelyrics', 'neo', 'nimrud', 'obmc', 'obta', 'ogsl

In [ ]:
rimanum = requests.get('http://oracc.org/rimanum/corpus.json')

In [ ]:
rimanum.content

b''